## lsing模型

#### 模拟退火

In [1]:
# 导入 kaiwu sdk
import kaiwu as kw

# 授权初始化代码
# 示例的user_id和sdk_code无效，需要替换成自己的用户ID和SDK授权码
kw.license.init(user_id="72205624632438786", sdk_code="sDsp8PTEJ5neUIR78DNMP7N5MtzyP7")

In [2]:
import numpy as np

# Import the plotting library
import matplotlib.pyplot as plt

# invert input graph matrix
# 根据优化目标，matrix就是lsing矩阵
matrix = -np.array([
                [0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
                [1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
                [0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
                [1, 0, 1, 0, 0, 1, 1, 0 ,1, 0],
                [1, 0, 1, 0, 0, 1, 0, 1, 0, 1],
                [0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
                [0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                [1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
                [1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
                [0, 0, 0, 0, 1, 1, 1, 0, 1, 0]])

In [3]:
worker = kw.classical.SimulatedAnnealingOptimizer(initial_temperature=100,
                                                  alpha=0.99,
                                                  cutoff_temperature=0.001,
                                                  iterations_per_t=10,
                                                  size_limit=100)
output = worker.solve(matrix)

In [4]:
print(output)

[[-1 -1 -1 -1  1 -1  1 -1  1 -1]
 [-1 -1 -1  1  1 -1 -1  1  1 -1]
 [-1 -1 -1  1  1 -1 -1 -1  1 -1]
 [-1 -1 -1  1  1 -1 -1  1  1  1]
 [-1  1 -1 -1  1 -1 -1  1  1  1]
 [-1 -1 -1  1  1 -1 -1 -1  1  1]
 [-1 -1  1 -1  1 -1  1 -1  1 -1]
 [-1 -1 -1  1  1  1  1  1 -1 -1]
 [ 1 -1 -1  1 -1 -1 -1  1 -1  1]
 [-1  1 -1  1  1  1 -1 -1 -1 -1]
 [-1  1 -1  1 -1 -1 -1 -1 -1  1]
 [-1 -1 -1  1  1 -1 -1  1 -1  1]
 [-1  1 -1 -1  1 -1 -1  1  1 -1]
 [-1  1 -1  1  1 -1  1 -1 -1 -1]
 [-1  1 -1  1  1  1 -1 -1 -1  1]
 [-1 -1  1  1  1 -1 -1  1 -1  1]
 [-1  1 -1  1 -1 -1 -1 -1  1 -1]
 [ 1 -1  1 -1 -1 -1  1  1  1 -1]
 [ 1 -1  1 -1 -1 -1  1 -1  1 -1]
 [ 1  1 -1  1  1 -1 -1 -1 -1 -1]
 [-1  1 -1  1  1  1 -1  1 -1  1]
 [-1  1 -1  1  1 -1  1  1 -1 -1]
 [-1  1 -1  1  1 -1  1  1  1  1]
 [-1 -1  1  1 -1  1 -1  1 -1 -1]
 [-1 -1  1 -1 -1  1  1 -1  1 -1]
 [ 1 -1 -1  1  1 -1  1 -1  1 -1]
 [-1 -1 -1  1  1 -1  1 -1  1 -1]
 [ 1 -1  1 -1  1  1  1  1  1 -1]
 [-1 -1  1 -1  1  1  1  1  1 -1]
 [-1  1 -1  1  1  1 -1 -1  1  1]
 [ 1  1 -1

In [5]:
opt = kw.sampler.optimal_sampler(matrix, output, 0)
best = opt[0][0]
max_cut = (np.sum(-matrix)-np.dot(-matrix,best).dot(best))/4
print("The obtained max cut is "+str(max_cut)+".")

The obtained max cut is 19.0.


## QUBO方法求解

In [6]:
import pandas as pd
import numpy as np

# Define the input adjacency matrix for the graph
adj_matrix = np.array([
                [0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
                [1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
                [0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
                [1, 0, 1, 0, 0, 1, 1, 0 ,1, 0],
                [1, 0, 1, 0, 0, 1, 0, 1, 0, 1],
                [0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
                [0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                [1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
                [1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
                [0, 0, 0, 0, 1, 1, 1, 0, 1, 0]])

# Get the number of nodes in the graph
num_nodes = len(adj_matrix)

# Create a QUBO variable array 'x' with 'num_nodes' variables
# each representing a node in the graph
x = kw.qubo.ndarray(num_nodes,'x',kw.qubo.binary)

# Define the objective function for the QUBO model of Max cut problem
obj = -x.T @ adj_matrix @ (1-x)

# parse QUBO
obj = kw.qubo.make(obj)

# Extract the QUBO matrix and store it in a pandas DataFrame
qubo_matrix = kw.qubo.qubo_model_to_qubo_matrix(obj)['qubo_matrix']

# Check whether the QUBO matrix satisfy the precision requirement
kw.qubo.check_qubo_matrix_bit_width(qubo_matrix)

# Save the QUBO matrix to a CSV file without including index or header
df = pd.DataFrame(qubo_matrix)
csv_file_path = 'qubo_matrix.csv'
df.to_csv(csv_file_path,index=False,header=False)

C:\Users\admin\AppData\Local\Temp\ipykernel_17916\4066116059.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) binary. (You can use the standard function `kw.qubo.Binary`.) -- Deprecated since version 1.0.0.
  x = kw.qubo.ndarray(num_nodes,'x',kw.qubo.binary)


In [7]:
print(qubo_matrix)

[[-5.  2.  0.  2.  2.  0.  0.  2.  2.  0.]
 [ 0. -5.  2.  0.  0.  2.  2.  2.  0.  0.]
 [ 0.  0. -4.  2.  2.  0.  0.  0.  2.  0.]
 [ 0.  0.  0. -5.  0.  2.  2.  0.  2.  0.]
 [ 0.  0.  0.  0. -5.  2.  0.  2.  0.  2.]
 [ 0.  0.  0.  0.  0. -5.  0.  0.  2.  2.]
 [ 0.  0.  0.  0.  0.  0. -3.  0.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.  0. -4.  2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -6.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -4.]]


In [11]:
# Convert to Ising model
obj_ising = kw.qubo.qubo_model_to_ising_model(obj)

# Extract the Ising matrix
ls_matrix2 = obj_ising.get_ising()["ising"]

print(ls_matrix2)

[[-0.   -0.25 -0.   -0.25 -0.25 -0.   -0.   -0.25 -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.   -0.   -0.25 -0.25 -0.25 -0.   -0.   -0.  ]
 [-0.   -0.25 -0.   -0.25 -0.25 -0.   -0.   -0.   -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.   -0.   -0.25 -0.25 -0.   -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.   -0.   -0.25 -0.   -0.25 -0.   -0.25 -0.  ]
 [-0.   -0.25 -0.   -0.25 -0.25 -0.   -0.   -0.   -0.25 -0.25 -0.  ]
 [-0.   -0.25 -0.   -0.25 -0.   -0.   -0.   -0.   -0.   -0.25 -0.  ]
 [-0.25 -0.25 -0.   -0.   -0.25 -0.   -0.   -0.   -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.25 -0.   -0.25 -0.   -0.25 -0.   -0.25 -0.  ]
 [-0.   -0.   -0.   -0.   -0.25 -0.25 -0.25 -0.   -0.25 -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]]


In [12]:
worker = kw.classical.SimulatedAnnealingOptimizer(initial_temperature=100,
                                                  alpha=0.99,
                                                  cutoff_temperature=0.001,
                                                  iterations_per_t=10,
                                                  size_limit=100)
output = worker.solve(ls_matrix2)

opt = kw.sampler.optimal_sampler(ls_matrix2, output, 0)
best = opt[0][0]
print(best)

[ 1 -1  1 -1 -1  1  1  1 -1  1 -1]


In [13]:
# Get the list of variable names
vars = obj_ising.get_variables()

# Substitute the spin vector and obtain the result dictionary
sol_dict = kw.qubo.get_sol_dict(best, vars)
print(sol_dict)

{'x[0]': 1.0, 'x[1]': 0.0, 'x[2]': 1.0, 'x[3]': 0.0, 'x[4]': 0.0, 'x[5]': 1.0, 'x[6]': 1.0, 'x[7]': 1.0, 'x[8]': 0.0, 'x[9]': 1.0}


In [14]:
# Get the numerical value matrix of x
x_val = kw.qubo.get_array_val(x, sol_dict)
print(x_val)

[1.0 0.0 1.0 0.0 0.0 1.0 1.0 1.0 0.0 1.0]


In [15]:
ans = -x_val.T @ adj_matrix @ (1-x_val)
print(ans)

-19.0


In [8]:
ls_matrix, _ = kw.qubo.qubo_matrix_to_ising_matrix(qubo_matrix)
print(ls_matrix)

[[-0.   -0.25 -0.   -0.25 -0.25 -0.   -0.   -0.25 -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.   -0.   -0.25 -0.25 -0.25 -0.   -0.   -0.  ]
 [-0.   -0.25 -0.   -0.25 -0.25 -0.   -0.   -0.   -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.   -0.   -0.25 -0.25 -0.   -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.   -0.   -0.25 -0.   -0.25 -0.   -0.25 -0.  ]
 [-0.   -0.25 -0.   -0.25 -0.25 -0.   -0.   -0.   -0.25 -0.25 -0.  ]
 [-0.   -0.25 -0.   -0.25 -0.   -0.   -0.   -0.   -0.   -0.25 -0.  ]
 [-0.25 -0.25 -0.   -0.   -0.25 -0.   -0.   -0.   -0.25 -0.   -0.  ]
 [-0.25 -0.   -0.25 -0.25 -0.   -0.25 -0.   -0.25 -0.   -0.25 -0.  ]
 [-0.   -0.   -0.   -0.   -0.25 -0.25 -0.25 -0.   -0.25 -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]]


In [9]:
worker = kw.classical.SimulatedAnnealingOptimizer(initial_temperature=100,
                                                  alpha=0.99,
                                                  cutoff_temperature=0.001,
                                                  iterations_per_t=10,
                                                  size_limit=100)
output = worker.solve(ls_matrix)
print(output)

opt = kw.sampler.optimal_sampler(ls_matrix, output, 0)
best = opt[0][0]
max_cut = (np.sum(-ls_matrix)-np.dot(-ls_matrix,best).dot(best))/4
print("The obtained max cut is "+str(max_cut)+".")

[[ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1  1]
 [-1 -1 -1 ...  1 -1 -1]
 ...
 [ 1 -1  1 ... -1  1  1]
 [ 1 -1  1 ... -1  1 -1]
 [ 1 -1  1 ...  1 -1 -1]]
The obtained max cut is 4.75.


In [10]:
max_cut*4

19.0

In [4]:
# 配置模拟退火求解器参数
optimizer = kw.classical.SimulatedAnnealingOptimizer(
    initial_temperature=100,  # 初始温度
    alpha=0.99,               # 温度衰减系数
    cutoff_temperature=1e-3,  # 停止温度
    iterations_per_t=1000,    # 每个温度的迭代次数
    size_limit=qubo_matrix.shape[0]  # QUBO 矩阵的维度
)

# 使用模拟退火算法求解 QUBO 模型
result = optimizer.solve(qubo_matrix)

# # 解码最优结果
# optimal_solution = result['solution']  # 最优解向量
# optimal_value = result['energy']       # 最优值（目标函数值）

# # 计算割权值
# cut_value = -optimal_value  # 对应最大割问题，目标函数为负，因此取负值

# print("最优解为:", optimal_solution)
# print("对应的最大割权值为:", cut_value)

# # 可视化分组结果
# group_a = [i for i, x in enumerate(optimal_solution) if x == 0]
# group_b = [i for i, x in enumerate(optimal_solution) if x == 1]

# print("分组 A 的节点:", group_a)
# print("分组 B 的节点:", group_b)
print(result)

[[-1 -1 -1  1  1 -1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1  1 -1]
 [-1 -1 -1  1 -1 -1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1 -1  1 -1 -1 -1]
 [-1  1 -1 -1 -1 -1  1 -1 -1  1]
 [-1  1 -1 -1  1 -1 -1 -1  1  1]
 [-1  1 -1 -1 -1 -1  1 -1 -1 -1]
 [-1  1 -1 -1  1 -1 -1 -1  1 -1]
 [-1  1 -1 -1  1 -1 -1 -1 -1  1]
 [-1  1 -1 -1 -1 -1 -1 -1 -1  1]]


In [6]:
# 提取对角线作为变量的初值
x = (np.diag(result) > 0).astype(int)  # 将对角线值转为二进制变量 (0 或 1)
print("解向量 x:", x)

# 计算目标值
objective_value = -x.T @ adj_matrix @ x
print("目标值:", objective_value)

解向量 x: [0 0 0 0 0 0 1 0 0 1]
目标值: -2
